In [ ]:
import os
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import EIDA
import matplotlib.pyplot as plt


tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # 사전 크기: 50257
model = GPT2LMHeadModel.from_pretrained("gpt2", device_map='auto') # layer 12개, d_model 768차원
tokenizer.pad_token = tokenizer.eos_token


# E2E NLG Challenge 데이터셋의 예시:
# name : Blue Spice | Type : coffee shop | area : city centre||A coffee shop in the city centre area called Blue Spice .
# '||' 이전 부분이 모델에 입력으로 주어지는 context(meaning representation 속성)
# '||' 이후 부분이 모델이 생성할 자연어 completion(human_reference 속성)
# E2E 폴더에 train.txt, valid.txt, test.txt 파일이 들어있어야 실행됨.
data = []
with open(os.path.join("E2E", "train.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
train_dataset = Dataset.from_list(data)

data = []
with open(os.path.join("E2E", "valid.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
valid_dataset = Dataset.from_list(data)

data = []
with open(os.path.join("E2E", "test.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
test_dataset = Dataset.from_list(data)

In [2]:
max_seq_len = 144 # context와 completion을 다 담기에 충분히 긴 길이. 128로 설정하면 test set에서 잘리는 행이 생김.

tokenized_data = []
for example in train_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)
    # GPT2의 input을 (meaning representation + BOS + ' ' + human reference + EOS)로 구성하는 것은 microsoft/LoRA 깃헙에서 입력을 구성한 방식을 그대로 따랐음.
    # GPT2에서는 BOS와 EOS 둘 다 50256번 토큰 <|endoftext|>으로 되어있고, PAD 토큰은 별도로 지정되어 있지 않음.
    # 현재 PAD 토큰을 50256으로 설정해서 사용하고 있음. padding한 위치들에는 labels값 -100을 설정해서 학습과정에 관여하지 않게 만들 것이므로 PAD 토큰을 50256으로 설정한 것이 학습에 영향을 미치지 않음.

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))] # context + completion + padding 구성으로 길이 max_seq_len만큼의 시퀀스

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]
    # labels에 -100이 들어있는 자리는 loss function이 무시하도록 설정되어 있다.(이것은 허깅페이스 transformers 패키지의 설정이 아니고 torch.nn.CrossEntropyLoss의 디폴트 설정임)
    # 모델이 context(meaning representation)를 입력받아 completion(human reference)을 생성하는 학습을 수행할 것이므로, labels에서 completion 부분의 토큰들만 남겨놓고 context 부분과 padding 부분은 -100으로 설정해주어야 의도한 학습을 수행하게 된다.
    
    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_train_dataset = Dataset.from_list(tokenized_data)


tokenized_data = []
for example in valid_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_valid_dataset = Dataset.from_list(tokenized_data)


tokenized_data = []
for example in test_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_test_dataset = Dataset.from_list(tokenized_data)

In [3]:
list_plane_dim = [4, 8, 16, 32, 64] # token representation들로 추정할 부분공간의 차원
dict_color = {4: 'violet', 8: 'blue', 16: 'green', 32: 'orange', 64: 'red'}

sample_size = 128 # 평면을 추정하는 데에 이용할 training set 표본 데이터 크기를 몇으로 할 건가

num_iter = 5 # 부분공간을 생성할 횟수
for t in range(num_iter):
    preprocessed_train_dataset = preprocessed_train_dataset.shuffle() # train set 순서 섞기
    input_ids = torch.tensor(preprocessed_train_dataset['input_ids'])
    labels = torch.tensor(preprocessed_train_dataset['labels'])

    EIDA.forward_gpt2_with_save(model, input_ids, labels, begin=0, end=sample_size, batch_size=2, max_length=max_seq_len, p=0.0075, dir=os.path.join("gpt-2", "sample"))
    # 부분공간 추정을 위한 token representation 표본 추출을 128개의 데이터에서 수행하고 "sample"라는 폴더에 저장해두기
    # 데이터(=문장) 각각에 대해, 표본추출은 모델을 통과하는 과정에서 파라미터의 input일 때의 위치 48곳, 파라미터의 output일 때의 위치 72곳에서 수행함. auto-regressive decoding loop를 돌려서 출력을 생성하니까 모델에 수십번 입력을 넣게 되는데, 토큰이 추출 위치에 오는 모든 경우마다 p=0.75% 확률로 추출을 수행함. 시퀀스 당 평균 8개를 추출하도록 하는 값임.

    for i in range(4*12):
        # 파라미터들의 input이 존재하는 latent space 48곳 중 한 곳(i번째)의 표본들을 sample_inputs 배열에 로드
        sample_inputs = []
        dir1 = os.path.join("gpt-2", "sample", "inputs", f"{i}")
        for f in os.listdir(dir1):
            if f.endswith(".pt"):
                sample_inputs.append(torch.load(os.path.join(dir1, f), weights_only=True))
        
        # 로드한 벡터들을 이용해서 PCA를 수행하여 평면을 추정하고, 결과는 [64, 768(W_fc1에서는 3072)] 크기의 텐서인 plane_inputs로 반환. plane_inputs[0:k, :]가 k차원 부분공간 추정의 결과임.
        plane_inputs = EIDA.PCA(sample_inputs, begin=0, end=len(sample_inputs), plane_dim=list_plane_dim[-1], device=model.device)
        os.makedirs(os.path.join("gpt-2", f"plane_{sample_size}_{t}", "inputs"), exist_ok=True)
        torch.save(plane_inputs, os.path.join("gpt-2", f"plane_{sample_size}_{t}", "inputs", f'{i}.pt'))
        del sample_inputs
    
    for i in range(6*12):
        # 파라미터들의 output이 존재하는 latent space 72곳 중 한 곳(i번째)의 표본들을 sample_delta_outputs 배열에 로드
        sample_delta_outputs = []
        dir1 = os.path.join("gpt-2", "sample", "delta_outputs", f"{i}")
        for f in os.listdir(dir1):
            if f.endswith(".pt"):
                sample_delta_outputs.append(torch.load(os.path.join(dir1, f), weights_only=True))
        
        # 로드한 벡터들을 이용해서 PCA를 수행하여 평면을 추정하고, 결과는 [64, 768(W_fc1에서는 3072)] 크기의 텐서인 plane_inputs로 반환. plane_inputs[0:k, :]가 k차원 부분공간 추정의 결과임.
        plane_delta_outputs = EIDA.PCA(sample_delta_outputs, begin=0, end=len(sample_delta_outputs), plane_dim=list_plane_dim[-1], device=model.device)
        os.makedirs(os.path.join("gpt-2", f"plane_{sample_size}_{t}", "delta_outputs"), exist_ok=True)
        torch.save(plane_delta_outputs, os.path.join("gpt-2", f"plane_{sample_size}_{t}", "delta_outputs", f'{i}.pt'))
        del sample_delta_outputs

In [4]:
# 추정된 4, 8, 16, 32, 64차원의 평면이 token representation의 분포를 얼마나 잘 설명하는지 측정하기 위해, 크기 1024의 표본을 다시 추출하여 이걸 평면에 정사영시켜보고 정사영과 원본벡터의 cosine similarity를 측정하는 과정.
preprocessed_train_dataset = preprocessed_train_dataset.shuffle() # train set 순서 섞기
input_ids = torch.tensor(preprocessed_train_dataset['input_ids'])
labels = torch.tensor(preprocessed_train_dataset['labels'])

EIDA.forward_gpt2_with_save(model, input_ids, labels, begin=0, end=sample_size, batch_size=2, max_length=max_seq_len, p=0.0075, dir=os.path.join("gpt-2", "sample"))


avg_input_proj_length = {k: [0.0 for i in range(4*12)] for k in list_plane_dim} # avg_input_proj_length[k][i]는 input token 표본 추출을 수행한 i번째 공간에서 추정된 k차원 평면에 다른 표본 토큰들을 정사영해보고 얻어진 원래 토큰과 정사영된 토큰 간의 cosine similarity 평균값
avg_output_proj_length = {k: [0.0 for i in range(6*12)] for k in list_plane_dim} # avg_output_proj_length[k][i]는 Δ(output) 표본 추출을 수행한 i번째 공간에서 추정된 k차원 평면에 다른 표본 토큰들을 정사영해보고 얻어진 원래 토큰과 정사영된 토큰 간의 cosine similarity 평균값


for t in range(num_iter):
    with torch.no_grad():
        for i in range(4*12):
            list_x = []
            dir2 = os.path.join("gpt-2", "sample", "inputs", f"{i}")
            for f in os.listdir(dir2):
                if f.endswith('.pt'):
                    list_x.append(torch.load(os.path.join(dir2, f), weights_only=True)) # 표본 한 개 로드

            plane_inputs = torch.load(os.path.join("gpt-2", f"plane_{sample_size}_{t}", "inputs", f"{i}.pt"), weights_only=True).to(model.device)
            X = torch.stack(list_x).to(model.device) # GPU 병렬연산을 위해 한 텐서로 합치기. 로드한 i번째 벡터는 X의 i행으로 있음.
        
            X = X / torch.norm(X, dim=1, keepdim=True) # 각 벡터를 길이 1로 정규화
            X = plane_inputs @ X.T # proj[i, j]는 로드한 j번째 표본벡터가 평면의 i번째 기저벡터인 plane_inputs[i, :] 방향의 성분을 얼마나 지녔는지를 의미. 이 시점부터는 개별 벡터의 정보가 X의 각 열로 있게 됨.
        
            for k in list_plane_dim:
                avg_input_proj_length[k][i] += torch.norm(X[0:k, :], dim=0).sum() / len(list_x) # 각 벡터를 k차원 평면에 정사영했을 때의 길이의 평균
            del X
            del list_x

        for i in range(6*12):
            list_y = []
            dir2 = os.path.join("gpt-2", "sample", "delta_outputs", f"{i}")
            for f in os.listdir(dir2):
                if f.endswith('.pt'):
                    list_y.append(torch.load(os.path.join(dir2, f), weights_only=True)) # 표본 한 개 로드
            
            plane_delta_outputs = torch.load(os.path.join("gpt-2", f"plane_{sample_size}_{t}", "delta_outputs", f"{i}.pt"), weights_only=True).to(model.device)
            Y = torch.stack(list_y).to(model.device) # GPU 병렬연산을 위해 한 텐서로 합치기. 로드한 i번째 벡터는 Y의 i행으로 있음.

            Y = Y / torch.norm(Y, dim=1, keepdim=True) # 각 벡터를 길이 1로 정규화
            Y = plane_delta_outputs @ Y.T # proj[i, j]는 로드한 j번째 표본벡터가 평면의 i번째 기저벡터인 plane_delta_outputs[i, :] 방향의 성분을 얼마나 지녔는지를 의미. 이 시점부터는 개별 벡터의 정보가 Y의 각 열로 있게 됨.

            for k in list_plane_dim:
                avg_output_proj_length[k][i] += torch.norm(Y[0:k, :], dim=0).sum() / len(list_y) # 각 벡터를 k차원 평면에 정사영했을 때의 길이의 평균
            del Y
            del list_y


for k in list_plane_dim:
    for i in range(4*12):
        avg_input_proj_length[k][i] /= num_iter
    for i in range(6*12):
        avg_output_proj_length[k][i] /= num_iter

In [ ]:
# 그래프 그려서 저장하기
os.makedirs(os.path.join("gpt-2", "graph"), exist_ok=True)

index_and_title = [(0, "Inputs of W_Q, W_K, W_V  (768 dimensions)"), (1, "Inputs of W_O  (768 dimensions)"), (2, "Inputs of W_fc1  (768 dimensions)"), (3, "Inputs of W_fc2  (3072 dimensions)")]
for index, title in index_and_title:
    plt.figure(figsize=(10, 6))
    for k in list_plane_dim:
        plt.plot(list(range(12)), [avg_input_proj_length[k][4*l+index].item() for l in range(12)], label=f"on {k}-dim", marker='o', linestyle='-', color=dict_color[k])
    plt.ylim(0.2, 1.0)
    plt.xticks(ticks=range(0, 12), labels=range(0, 12))
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.xlabel("layers")
    plt.ylabel("average cosine similarity")
    plt.title(title)
    plt.legend()

    plt.savefig(os.path.join("gpt-2", "graph", f"{sample_size}_input_{index}.png"), dpi=300, bbox_inches='tight')
    plt.show()


index_and_title = [(0, "Δ(outputs) of W_Q  (768 dimensions)"), (1, "Δ(outputs) of W_K  (768 dimensions)"), (2, "Δ(outputs) of W_V  (768 dimensions)"), (3, "Δ(outputs) of W_O  (768 dimensions)"), (4, "Δ(outputs) of W_fc1  (3072 dimensions)"), (5, "Δ(outputs) of W_fc2  (768 dimensions)")]
for index, title in index_and_title:
    plt.figure(figsize=(10, 6))
    for k in list_plane_dim:
        plt.plot(list(range(12)), [avg_output_proj_length[k][6*l+index].item() for l in range(12)], label=f"on {k}-dim", marker='o', linestyle='-', color=dict_color[k])
    plt.ylim(0.2, 1.0)
    plt.xticks(ticks=range(0, 12), labels=range(0, 12))
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.xlabel("layers")
    plt.ylabel("average cosine similarity")
    plt.title(title)
    plt.legend()

    plt.savefig(os.path.join("gpt-2", "graph", f"{sample_size}_delta_output_{index}.png"), dpi=300, bbox_inches='tight')
    plt.show()